In [ ]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nlp=spacy.load("en_core_web_sm")

In [ ]:
data=pd.read_csv("/content/Reviews.csv", on_bad_lines="skip",engine="python")
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
data['Text']

,Text
0,I have bought several of the Vitality canned d...
1,Product arrived labeled as Jumbo Salted Peanut...
2,This is a confection that has been around a fe...
3,If you are looking for the secret ingredient i...
4,Great taffy at a great price. There was a wid...
...,...
14204,I was excited to be able to give these to my d...
14205,My mountain feist (a breed that is similar to ...
14206,The bottom line is that I recommend these trea...
14207,These lamb and barley treats have some excelle...


In [ ]:
data.isna().sum()

,0
Id,0
ProductId,0
UserId,0
ProfileName,1
HelpfulnessNumerator,0
HelpfulnessDenominator,0
Score,0
Time,0
Summary,0
Text,0


In [ ]:
data.dropna(inplace=True)

In [ ]:
processed_data=data[:1000]

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


In [ ]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def cleaned_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

data['cleaned_text'] = data['Text'].astype(str).apply(cleaned_text)
print(data['cleaned_text'].head())


0    bought several vitality canned dog food produc...
1    product arrived labeled jumbo salted peanutsth...
2    confection around centuries light pillowy citr...
3    looking secret ingredient robitussin believe f...
4    great taffy great price wide assortment yummy ...
Name: cleaned_text, dtype: object


In [ ]:
def pos(text):
  doc=nlp(text)
  return [(token.text,token.pos_,token.tag_) for token in doc]

In [ ]:
data['POS_TAGGING'] = data['cleaned_text'].head(1000).apply(pos)
print(data[['POS_TAGGING']].head())


                                         POS_TAGGING
0  [(bought, VERB, VBD), (several, ADJ, JJ), (vit...
1  [(product, NOUN, NN), (arrived, VERB, VBD), (l...
2  [(confection, NOUN, NN), (around, ADP, IN), (c...
3  [(looking, VERB, VBG), (secret, ADJ, JJ), (ing...
4  [(great, ADJ, JJ), (taffy, ADJ, JJ), (great, A...


In [ ]:
def ner(text):
  doc=nlp(text)
  pos_tags=[(token.text,token.pos_,token.tag_) for token in doc]
  entities=[(ent.text,ent.label_) for ent in doc.ents]
  return {"ner":entities}

In [ ]:
data['NER'] = data['cleaned_text'].head(1000).apply(ner)
print(data[['NER']].head())


                                                 NER
0  {'pos': [('bought', 'VERB', 'VBD'), ('several'...
1  {'pos': [('product', 'NOUN', 'NN'), ('arrived'...
2  {'pos': [('confection', 'NOUN', 'NN'), ('aroun...
3  {'pos': [('looking', 'VERB', 'VBG'), ('secret'...
4  {'pos': [('great', 'ADJ', 'JJ'), ('taffy', 'AD...
